# Jeszcze jedna
## Flask ciąg dalszy

### Marcin Jaroszewski
### 15.III.2018, Python Level UP

![Logo kursu Python Level Up](https://raw.githubusercontent.com/daftcode/python_levelup_2018/master/logo.png)

## Tutoriale flask
- https://www.tutorialspoint.com/flask/index.htm
- http://flask.pocoo.org/docs/0.12/tutorial/

# Po co w ogóle są webaplikacje?

https://pages.apigee.com/rs/apigee/images/api-design-ebook-2012-03.pdf

# HTML "dynamiczny"

## Podejście prymitywne/chałupnicze

Podstawowa aplikacja flask:
```python
from flask import Flask, request
app = Flask(__name__)

@app.route("/")
def hello():
	return 'hello from flask'

if __name__ == '__main__':
    app.run(debug=True)
```

### Pobieranie danych z querystring
Standardowe zapytanie z przeglądarki to GET.

GET z założenia nie przesyła danych do serwera na "trwałe" (do tego służy POST).

Ale GET ma możliwość przesłania pewnej ilości danych do serwera za pomocą **querystring**.

Te dane nie powinny zostać zapisane w serwerze na stałe, ale mogą posłużyć do wygenerowania odpowiedzi.

```python
@app.route("/request_query_string_discovery")
def print_request():
    print(dir(request))
    print('request.args:', request.args)
    print('type(request.args):', type(request.args))
    print('request.query_string:', request.query_string)
    return ''
```

To co nas interesuje to: `request.args`
```python
type(request.args): <class 'werkzeug.datastructures.ImmutableMultiDict'>
```
dokumentacja:
1. http://werkzeug.pocoo.org/docs/0.14/datastructures/#werkzeug.datastructures.ImmutableMultiDict
2. http://werkzeug.pocoo.org/docs/0.14/datastructures/#werkzeug.datastructures.MultiDict

### Proste wstawianie danych od użytkownika w nasz HTML

http://127.0.0.1:5000/request_query_string_based_response?ala=ma_kota

### Złośliwy użytkownik i znaczniki HTML w inpucie

Pierwszy złośliwy url ze wstrzyknięciem html: http://127.0.0.1:5000/request_query_string_based_response?+=%3C%21DOCTYPE+html%3E%3Chtml%3E%3Chead%3E%3Ctitle%3EPage+Title%3C%2Ftitle%3E%3C%2Fhead%3E%3Cbody%3E%3Ch1%3EHACKED%21%21%21%3C%2Fh1%3E%3C%2Fbody%3E%3C%2Fhtml%3E

```html
<!DOCTYPE html><html><head><title>Page Title</title></head><body><h1>HACKED!!!</h1></body></html>
```

In [1]:
evil_input = '<!DOCTYPE html><html><head><title>Page Title</title></head><body><h1>HACKED!!!</h1></body></html>'
import urllib.parse
query_string = urllib.parse.urlencode({' ': evil_input})
print(query_string)

+=%3C%21DOCTYPE+html%3E%3Chtml%3E%3Chead%3E%3Ctitle%3EPage+Title%3C%2Ftitle%3E%3C%2Fhead%3E%3Cbody%3E%3Ch1%3EHACKED%21%21%21%3C%2Fh1%3E%3C%2Fbody%3E%3C%2Fhtml%3E


Drugi złośliwy url ze wstrzyknięciem html i javascript: http://127.0.0.1:5000/request_query_string_based_response?+=%3C%21DOCTYPE+html%3E%3Chtml%3E%3Chead%3E%3Ctitle%3EPage+Title%3C%2Ftitle%3E%3C%2Fhead%3E%3Cscript%3Ewindow.location.replace%28%22https%3A%2F%2Fhaveibeenpwned.com%2F%22%29%3B%3C%2Fscript%3E%3Cbody%3E%3C%2Fbody%3E%3C%2Fhtml%3E

```html
<!DOCTYPE html><html><head><title>Page Title</title></head><script>window.location.replace("https://haveibeenpwned.com/");</script><body></body></html>
```

In [2]:
more_evil_input = '<!DOCTYPE html><html><head><title>Page Title</title></head><script>window.location.replace("https://haveibeenpwned.com/");</script><body></body></html>'
query_string = urllib.parse.urlencode({' ': more_evil_input})
print(query_string)

+=%3C%21DOCTYPE+html%3E%3Chtml%3E%3Chead%3E%3Ctitle%3EPage+Title%3C%2Ftitle%3E%3C%2Fhead%3E%3Cscript%3Ewindow.location.replace%28%22https%3A%2F%2Fhaveibeenpwned.com%2F%22%29%3B%3C%2Fscript%3E%3Cbody%3E%3C%2Fbody%3E%3C%2Fhtml%3E


Ataków zbliżonych do przedstawionych jest bardzo wiele. Ciężko się zamodzielnie przed nimi bronić. W dodadtku samodzielne składanie html też nie jest takie łatwe. Na szczęście Flask i wiele innych frameworków webowych oferuje nam system templatek.

# Templatki

## Jinja2

Dokumentacja: http://jinja.pocoo.org/docs/2.10/

```html
<!DOCTYPE html>
<html lang="en">
<head>
    <title>Wyrenderowane z templatki</title>
</head>
<body>
    {% for key, value in query_string_data|dictsort %}
        <h1>{{ key }}: {{ value }}</h1>
    {% endfor %}
</body>
</html>
```

Templatki we Flask same robią autoescape danych, które mogą popsuć stronę.

# Routing

### Pobieranie danych z url

```python
@app.route("/simple_path_tmpl/<sample_variable>")
def simple_path_tmpl(sample_variable):
    print(sample_variable)
    print(type(sample_variable))
    print(app.url_map)
    return render_template(
        'route_description_tmpl.html',
        value=sample_variable,
        my_type=type(sample_variable),
        my_id=id(sample_variable),
    )
```

### Pobieranie danych z url z automatyczną konwersją
```python
@app.route("/simple_path_int/<int:sample_variable>")
def simple_path_int(sample_variable):
    print(sample_variable)
    print(type(sample_variable))
    print(app.url_map)
    return render_template(
        'route_description_tmpl.html',
        value=sample_variable,
        my_type=type(sample_variable),
        my_id=id(sample_variable),
    )
```

### Obsługa dowolnej ścieżki
```python
@app.route("/path/<path:my_path>")
def path_all(my_path):
    print(my_path)
    print(type(my_path))
    print(app.url_map)
    return render_template(
        'route_description_tmpl.html',
        value=my_path,
        my_type=type(my_path),
        my_id=id(my_path),
    )
```

# Obsługa różnych metod

Metody HTTP służą do tego, żeby dla tego samego url zachowanie/odpowiedź serwera były inne. W miarę możliwości zgodne ze znaczeniem użytej metody.

## GET - pobieranie zasobu
```python
@app.route("/person/<person_name>")
def person_info(person_name):
    global persons
    # do samodzielnego zastanowienia się, czy to bezpieczne
    person = persons.get(person_name)
    return render_template(
        'person_tmpl.html',
        name=person.get('name'),
        surname=person.get('surname'),
        occupation=person.get('occupation'),
    )
```

## Postman i inne podobne
Do wysyłania requestów innych niż GET i lepszej kontroli na requestami.
Można też napisać sobie skrypty w python (np z użyciem biblioteki requests).
- https://www.getpostman.com/

## POST - dodawanie zasobu
```python
@app.route("/person/<person_name>", methods=['GET', 'POST'])
def person_info(person_name):
	if request.method == 'GET':
		return get_person_info(person_name)
	elif request.method == 'POST':
		return post_person_info(person_name)
```

```python
def post_person_info(person_name):
	data = request.get_json()
	new_person = {
		'name': data.get('name'),
		'surname': data.get('surname'),
		'occupation': data.get('occupation')
	}
	global persons
	persons[data.get('name')] = new_person
	return 'OK'
```

# Cookies

## Co to są ciasteczka

## Ustawianie i odczyt ciasteczka

```python
@app.route("/my_cookies")
def cookies():
	cookie_secret = request.cookies.get('cookie_secret')
	resp = make_response(
		render_template(
			'cookies_tmpl.html', cookie_secret=cookie_secret
		)
	)
	resp.set_cookie('cookie_secret', 'I am cookie')
	return resp
```

# Bezpieczeństwo

Każdy może nam w naszej aplikacji czytać, dodawać, zmieniać, kasować dowolne zasoby. A to raczej niedobrze.

# BasicAuth

## Logowanie przez ciasteczko